[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/GP/PlanejamentoAgregado_pyomo.ipynb)

# Modelagem e Gestão de Processos

TEP - UFF


**Professor: Diogo Ferreira de Lima Silva**

## Um Problema de Planejamento Agregado

In [142]:
#Execute esse bloco caso esteja executando no Google Colab
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

^C


'apt-get' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [117]:
import pyomo.environ as pe
import pandas as pd

## Etapa 1: Coleta de Dados do Problema

In [123]:
periodos = [1,2,3,4]
decisoes = ['Regular', 'HoraExtra', 'Sub']

In [124]:
capacidades = pd.DataFrame({'Regular': [52400, 42900, 62300, 62300],
                        'HoraExtra': [8400 for i in range(4)],
                        'Sub': [4200 for i in range(4)]
                        }, index = periodos)
capacidades

,Regular,HoraExtra,Sub
1,52400,8400,4200
2,42900,8400,4200
3,62300,8400,4200
4,62300,8400,4200


In [125]:
capacidades_dict = capacidades.stack().to_dict()
capacidades_dict

{(1, 'Regular'): 52400,
 (1, 'HoraExtra'): 8400,
 (1, 'Sub'): 4200,
 (2, 'Regular'): 42900,
 (2, 'HoraExtra'): 8400,
 (2, 'Sub'): 4200,
 (3, 'Regular'): 62300,
 (3, 'HoraExtra'): 8400,
 (3, 'Sub'): 4200,
 (4, 'Regular'): 62300,
 (4, 'HoraExtra'): 8400,
 (4, 'Sub'): 4200}

In [126]:
custos = {'Regular': 480, 'HoraExtra': 600, 'Sub': 750}
custos

{'Regular': 480, 'HoraExtra': 600, 'Sub': 750}

In [127]:
demandas = {1: 52400, 2: 45800, 3: 65000, 4: 87600}
demandas

{1: 52400, 2: 45800, 3: 65000, 4: 87600}

## Etapa 2: Modelagem

### Criando o modelo

Para instanciar o modelo, basta usar a função ConcreteModel():

In [136]:
M = pe.ConcreteModel()

### Conjuntos

Criando os nossos conjuntos:
- Um conjunto para os períodos, chamado de I
- Um conjunto para os tipos de produção, chamado de J

In [137]:
M.I = pe.Set(initialize = periodos)
M.J = pe.Set(initialize = decisoes)

### Parâmetros

Nossos parâmetros incluem as demandas, capacidades de produção e os custos

In [138]:
M.demandas = pe.Param(M.I, initialize = demandas)
M.capacidades = pe.Param(M.I, M.J, initialize =  capacidades_dict)
M.custos_prod = pe.Param(M.J, initialize = custos)
M.custo_manter = pe.Param(initialize = 225)

### Variáveis de Decisão

In [139]:
M.x = pe.Var(M.I, M.J, within = pe.NonNegativeReals)
M.estoque_inicial = pe.Var(M.I, within = pe.NonNegativeReals)
M.estoque_final = pe.Var(M.I, within = pe.NonNegativeReals)

### Função Objetivo

In [140]:
def custo_total(M):
    return sum(M.x[i,j] * M.custos_prod[j] for i in M.I for j in M.J) + sum(M.estoque_final[i] * M.custo_manter for i in M.I)

M.z = pe.Objective(rule = custo_total, sense = pe.minimize)

### Restrições

In [141]:
# Restrições de demanda
M.R_demanda = pe.ConstraintList()
for i in M.I:
    M.R_demanda.add(expr = M.estoque_inicial[i] + sum(M.x[i, j] for j in M.J) >= M.demandas[i])

# Restrições de capacidade
M.R_capacidade = pe.ConstraintList()
for i in M.I:
    for j in M.J:
        M.R_capacidade.add(expr = M.x[i,j] <= M.capacidades[i,j])

# Lógica dos Estoques
M.R_estoquefinal = pe.ConstraintList()
M.R_estoqueinicial = pe.ConstraintList()
for i in M.I:    
    M.R_estoquefinal.add(expr = M.estoque_inicial[i] + sum(M.x[i, j] for j in M.J) - M.demandas[i] == M.estoque_final[i])
    if i > 1: 
        M.R_estoqueinicial.add( expr = M.estoque_inicial[i] == M.estoque_final[i-1]) # Estoque inicial [i] == estoque final [i-1]

# Estoque inicial do primeiro período deve ser 5000
M.R_estoquenicial_1 = pe.Constraint(expr = M.estoque_inicial[1] == 5000)

# Estoque final do último período deve ser 8000
M.estoque_final_n = pe.Constraint(expr = M.estoque_final[4] == 8000)

### Resolvendo o Modelo e obtendo a solução alcançada

Para resolver o modelo, precisamos de um solver. Usaremos aqui o glpk

In [ ]:
# RESOLUÇÃO DO MODELO
pe.SolverFactory('glpk').solve(M)

{'Problem': [{'Name': 'unknown', 'Lower bound': 136534500.0, 'Upper bound': 136534500.0, 'Number of objectives': 1, 'Number of constraints': 26, 'Number of variables': 21, 'Number of nonzeros': 57, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.11109590530395508}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [116]:
print(f"z= {pe.value(M.z)}")

for k in M.x.keys(): 
    if pe.value(M.x[k]) > 0:
        print (f"x_{k} = {pe.value(M.x[k])}")


for k in M.estoque_inicial.keys(): 
    if pe.value(M.estoque_inicial[k]) > 0:
        print (f"estoque_inicial_{k} = {pe.value(M.estoque_inicial[k])}")

for k in M.estoque_final.keys(): 
    if pe.value(M.estoque_final[k]) > 0:
        print (f"estoque_final_{k} = {pe.value(M.estoque_final[k])}")

z= 137209500.0
x_(1, 'Regular') = 52400.0
x_(2, 'Regular') = 42900.0
x_(2, 'HoraExtra') = 8400.0
x_(2, 'Sub') = 300.0
x_(3, 'Regular') = 62300.0
x_(3, 'HoraExtra') = 8400.0
x_(3, 'Sub') = 4200.0
x_(4, 'Regular') = 62300.0
x_(4, 'HoraExtra') = 8400.0
x_(4, 'Sub') = 4200.0
estoque_inicial_1 = 5000.0
estoque_inicial_2 = 5000.0
estoque_inicial_3 = 10800.0
estoque_inicial_4 = 20700.0
estoque_final_1 = 5000.0
estoque_final_2 = 10800.0
estoque_final_3 = 20700.0
estoque_final_4 = 8000.0
